In [75]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import folium as fol
from folium.plugins import MarkerCluster
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dc-metro-crime-data/DC_Crime_Data.csv


In [76]:
dc_crime = pd.read_csv('/kaggle/input/dc-metro-crime-data/DC_Crime_Data.csv')

In [77]:
dc_crime = dc_crime.fillna(0)

In [78]:
dc_crime.head()

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID
0,-77.013122,38.900761,21172055,2021/11/25 00:45:08+00,EVENING,OTHERS,THEFT F/AUTO,800 - 899 BLOCK OF NEW JERSEY AVENUE NW,398862.0,136989.0,...,004702 1,4702.0,Precinct 1,38.900753,-77.013119,MOUNT VERNON TRIANGLE CID,2021/11/25 00:27:00+00,2021/11/25 00:36:00+00,520976088,0.0
1,-77.036500,38.921854,21057211,2021/05/04 16:21:16+00,DAY,OTHERS,THEFT F/AUTO,2400 - 2479 BLOCK OF 16TH STREET NW,396835.0,139331.0,...,003801 1,3801.0,Precinct 24,38.921846,-77.036498,0,2021/05/03 16:40:06+00,2021/05/03 17:45:41+00,520976090,0.0
2,-77.042746,38.934175,21077218,2021/06/10 14:39:59+00,DAY,OTHERS,BURGLARY,1800 - 1899 BLOCK OF NEWTON STREET NW,396294.0,140699.0,...,002704 1,2704.0,Precinct 40,38.934167,-77.042744,0,2021/06/09 13:00:29+00,2021/06/10 14:40:09+00,520976100,0.0
3,-77.034578,38.906108,21078115,2021/06/12 05:25:32+00,MIDNIGHT,GUN,ROBBERY,1200 - 1219 BLOCK OF 15TH STREET NW,397001.0,137583.0,...,010100 1,10100.0,Precinct 17,38.906100,-77.034576,DOWNTOWN,2021/06/12 03:05:12+00,2021/06/12 03:45:56+00,520976101,0.0
4,-76.932725,38.898797,21077859,2021/06/11 20:51:35+00,EVENING,OTHERS,MOTOR VEHICLE THEFT,4800 - 4899 BLOCK OF NANNIE HELEN BURROUGHS AV...,405836.0,136773.0,...,007804 1,7804.0,Precinct 98,38.898789,-76.932723,0,2021/06/11 19:20:36+00,2021/06/11 19:22:18+00,520976110,0.0


In [79]:
dc_crime.shape

(292906, 25)

In [80]:
dc_crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292906 entries, 0 to 292905
Data columns (total 25 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   X                     292906 non-null  float64
 1   Y                     292906 non-null  float64
 2   CCN                   292906 non-null  int64  
 3   REPORT_DAT            292906 non-null  object 
 4   SHIFT                 292906 non-null  object 
 5   METHOD                292906 non-null  object 
 6   OFFENSE               292906 non-null  object 
 7   BLOCK                 292906 non-null  object 
 8   XBLOCK                292906 non-null  float64
 9   YBLOCK                292906 non-null  float64
 10  WARD                  292906 non-null  float64
 11  ANC                   292906 non-null  object 
 12  DISTRICT              292906 non-null  float64
 13  PSA                   292906 non-null  float64
 14  NEIGHBORHOOD_CLUSTER  292906 non-null  object 
 15  

In [81]:
dc_crime.head()

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID
0,-77.013122,38.900761,21172055,2021/11/25 00:45:08+00,EVENING,OTHERS,THEFT F/AUTO,800 - 899 BLOCK OF NEW JERSEY AVENUE NW,398862.0,136989.0,...,004702 1,4702.0,Precinct 1,38.900753,-77.013119,MOUNT VERNON TRIANGLE CID,2021/11/25 00:27:00+00,2021/11/25 00:36:00+00,520976088,0.0
1,-77.036500,38.921854,21057211,2021/05/04 16:21:16+00,DAY,OTHERS,THEFT F/AUTO,2400 - 2479 BLOCK OF 16TH STREET NW,396835.0,139331.0,...,003801 1,3801.0,Precinct 24,38.921846,-77.036498,0,2021/05/03 16:40:06+00,2021/05/03 17:45:41+00,520976090,0.0
2,-77.042746,38.934175,21077218,2021/06/10 14:39:59+00,DAY,OTHERS,BURGLARY,1800 - 1899 BLOCK OF NEWTON STREET NW,396294.0,140699.0,...,002704 1,2704.0,Precinct 40,38.934167,-77.042744,0,2021/06/09 13:00:29+00,2021/06/10 14:40:09+00,520976100,0.0
3,-77.034578,38.906108,21078115,2021/06/12 05:25:32+00,MIDNIGHT,GUN,ROBBERY,1200 - 1219 BLOCK OF 15TH STREET NW,397001.0,137583.0,...,010100 1,10100.0,Precinct 17,38.906100,-77.034576,DOWNTOWN,2021/06/12 03:05:12+00,2021/06/12 03:45:56+00,520976101,0.0
4,-76.932725,38.898797,21077859,2021/06/11 20:51:35+00,EVENING,OTHERS,MOTOR VEHICLE THEFT,4800 - 4899 BLOCK OF NANNIE HELEN BURROUGHS AV...,405836.0,136773.0,...,007804 1,7804.0,Precinct 98,38.898789,-76.932723,0,2021/06/11 19:20:36+00,2021/06/11 19:22:18+00,520976110,0.0


In [84]:
dc_crime_2023 = dc_crime[dc_crime['REPORT_DAT'].str.contains('2023', case=True, na=False)]

In [85]:
dc_crime_2023.shape

(34220, 25)

In [86]:
dc_crime_2023.head()

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID
58112,-77.028088,38.906443,22190292,2023/01/01 00:23:54+00,EVENING,OTHERS,THEFT F/AUTO,1200 - 1299 BLOCK OF 12TH STREET NW,397564.0,137620.0,...,005004 1,5004.0,Precinct 17,38.906435,-77.028085,0,2022/12/30 19:00:00+00,2022/12/30 21:00:00+00,521799145,0.0
59202,-77.001491,38.953424,22190321,2023/01/01 02:59:06+00,EVENING,OTHERS,THEFT/OTHER,5200 - 5299 BLOCK OF 3RD STREET NE,399871.0,142835.0,...,009508 3,9508.0,Precinct 66,38.953417,-77.001488,0,2023/01/01 00:53:00+00,2023/01/01 01:25:00+00,521803022,0.0
59961,-77.005894,38.905644,22190293,2023/01/01 00:37:44+00,EVENING,OTHERS,MOTOR VEHICLE THEFT,M STREET NE AND 1ST STREET NE,399489.0,137531.0,...,010603 2,10603.0,Precinct 144,38.905636,-77.005891,NOMA,2022/12/31 23:56:00+00,2023/01/01 00:25:00+00,521804811,0.0
59962,-77.014601,38.973548,22190373,2023/01/01 04:31:23+00,MIDNIGHT,OTHERS,THEFT/OTHER,6900 - 6999 BLOCK OF WILLOW STREET NW,398735.0,145069.0,...,001702 2,1702.0,Precinct 63,38.973540,-77.014598,0,2023/01/01 02:57:00+00,2023/01/01 04:22:00+00,521804812,0.0
61746,-77.021921,38.911723,22190352,2023/01/01 03:00:17+00,EVENING,GUN,ROBBERY,1600 - 1623 BLOCK OF 7TH STREET NW,398099.0,138206.0,...,004901 2,4901.0,Precinct 18,38.911715,-77.021919,0,2023/01/01 02:00:00+00,2023/01/01 02:56:00+00,521808880,0.0


In [99]:
#inserts map, and zooms in on The Capitol Building: The Center of DC
dc_map = fol.Map(location=[38.8899, -77.0091], zoom_start=12)

In [100]:
#add WMATA routes
wmata_routes = "https://opendata.dc.gov/api/download/v1/items/ead6291a71874bf8ba332d135036fbda/geojson?layers=58"

fol.GeoJson(wmata_routes, name="WMATA Metro Routes").add_to(dc_map)

In [101]:
marker_cluster = MarkerCluster().add_to(dc_map)

In [102]:
#insert points
for _, row in dc_crime_2023.iterrows():
    fol.Marker(
        location=[row['Y'], row['X']],  # Use correct lat/lon
        popup=f"Offense: {row['OFFENSE']}<br>Date: {row['REPORT_DAT']}",
        icon=fol.Icon(color='blue', icon='info-sign')
    ).add_to(marker_cluster)

In [103]:
#saves Map in HTML file that can be viewed externally (without crashing Kaggle)
dc_map.save('DC_crime_map_with_metro.html')

In [98]:
dc_crime_2023['OFFENSE'].value_counts()

OFFENSE
THEFT/OTHER                   13259
THEFT F/AUTO                   7748
MOTOR VEHICLE THEFT            6782
ROBBERY                        3462
ASSAULT W/DANGEROUS WEAPON     1404
BURGLARY                       1090
HOMICIDE                        274
SEX ABUSE                       190
ARSON                            11
Name: count, dtype: int64

In [ ]:
dc_crime_2023['OFFENSE']